In [ ]:
import requests
import json
import time
import re

In [ ]:
def get_attribute(data, attribute, default_value=''):
    if data:
        if attribute in data:
            return data[attribute]
        else:
            return default_value
    else:
        return default_value

def get_items_metadata(jsonObj):
    items = dict()
    if "_embedded" in jsonObj:
        if "items" in jsonObj["_embedded"]:
            for item in jsonObj["_embedded"]["items"]:
                sesamid = item["id"]
                items[sesamid] = {}
                if 'metadata' in item:
                    items[sesamid]['creators'] = get_attribute(item['metadata'], 'creators')
                    items[sesamid]['title'] = get_attribute(item['metadata'], 'title')
                    items[sesamid]['identifiers'] = get_attribute(item['metadata'], 'identifiers')
                if 'accessInfo' in item:
                    items[sesamid]['isDigital'] = get_attribute(item['accessInfo'], 'isDigital')                
        return items
    else:
        return None

In [ ]:
query = {"""add more filter types from list below in the brackets"""
        'filter':['mediatype:privatarkivmateriale'],
        # filter types: 
            #'mediatype:privatarkivmateriale'
            # 'api_namerecipients:"garborg, hulda"'
            #'api_title:"brevs. 929"'
            # 'api_namecreators:"Backer, Harriet"'
            # 'api_namerecipient:"Undset, Sigrid"'
            # 'date:[19000101 TO 19991231]'
        #'&aggs=mediatype&aggs=year:9999&snippets=privatarkivmateriale&fragments=2&fragSize=500&size=50&profile=nbdigital
        #'sort' :'firstDigitalContentTime,desc',
        'digitalAccessibleOnly' :'true',
        # size of each page (max appears to be 50, getting a 500 above that)
        'size': 50,
        #'snippets': 'aviser',
        #'fragments':10
        # få alle treff
        'profile': 'nbdigital',
        'scroll': True
    }

In [ ]:
itemDict = dict()
next_page = True
counter = 1
r = requests.get("https://api.nb.no/catalog/v1/items", params = query)
obj = r.json()
items = get_items_metadata(obj)
itemDict.update(items)
print(obj["page"]["totalElements"], "elementer totalt i", obj["page"]["totalPages"], "bolker")

while next_page == True:
    try:
        link = obj["_links"]["next"]["href"]
    except:
        next_page = False
        continue
        
    print(counter)
    counter += 1
    r = requests.get(link)
    obj = r.json()
    items = get_items_metadata(obj)
    itemDict.update(items)
    time.sleep(0)

In [ ]:
with open('sesamids.txt', 'w') as file:
    for key in itemDict:
        try:
            file.write(key + '\n')
        except:
            pass

In [ ]:
sesamids = []
for key in itemDict:
    sesamids.append(key)

In [ ]:
len(sesamids)

In [ ]:
sesamids